In [1]:
import pymongo
import pandas as pd
import numpy as np
import math
from datetime import datetime

# import dask.dataframe as dd ## 대규모 데이터 처리
# import vaex ## 대규모 데이터 처리

# from sklearn.linear_model import LinearRegression ## likes, dislikes 대체
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

import os
from dotenv import load_dotenv

c:\Users\kwan\AppData\Local\anaconda3\envs\awake_venv\lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
file_path = 'C:/py_src/awake/data/'

In [3]:
# 계정 / 콘텐츠 분석 데이터셋 불러오기
merge_df_users_fin = pd.read_csv(file_path + 'merge_df_users_final2.csv', low_memory=False)
# youtube_videos = pd.read_csv(file_path + 'youtube_videos.csv')

## 계정 데이터 분석

### 데이터 분할
- 미래를 예측하기 위한 모델이므로 시간순으로 데이터 분할
- 시작날짜 : 2023-03-26
- 종료날짜 : 2024-05-06
- 전체 기간의 80% 날짜 : 2024-02-11

In [5]:
# 컬럼 정리
unique_col = merge_df_users_fin.columns[:11]
x_col = merge_df_users_fin.columns[11:].drop('subscribers_count') ## y값 제거

In [6]:
# 데이터 분할
train_data = merge_df_users_fin[merge_df_users_fin['date'] <= '2024-02-11']
test_data = merge_df_users_fin[merge_df_users_fin['date'] > '2024-02-11']

In [7]:
# 불균형 확인
print(train_data.shape)
print(test_data.shape)

(84473, 56)
(21210, 56)


### 모델 기법 적용

In [8]:
# 각 모델 정의
rf_model = RandomForestRegressor(random_state=42)
gbm_model = GradientBoostingRegressor(random_state=42)
lgbm_model = LGBMRegressor(random_state=42)
xgb_model = xgb.XGBRegressor(random_state=42)

# 모델 리스트
models = {
    'RandomForest': rf_model,
    'GBM': gbm_model,
    'LightGBM': lgbm_model,
    'XGBoost': xgb_model
}

In [9]:
# # 모델별 교차 검증 결과 저장
# results = {}

# for model_name, model in models.items():
#     print(f"\n{model_name} 모델 성능 평가 중...")
    
#     # 교차 검증
#     cv_scores = cross_val_score(model, train_data[x_col], train_data['subscribers_count'], cv=5, scoring='neg_mean_squared_error')
    
#     # 평균 RMSE 계산
#     rmse_scores = np.sqrt(-cv_scores)  # neg_mean_squared_error는 음수이므로 양수로 변환 후 제곱근
#     mean_rmse = rmse_scores.mean()
#     print(f"{model_name} 교차 검증 평균 RMSE: {mean_rmse}")
    
#     # 결과 저장
#     results[model_name] = mean_rmse

### 모델 성능 평가

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

for model in models:
    # 모델 정의 및 학습
    models[model].fit(train_data[x_col], train_data['subscribers_count'])

    # 예측
    y_pred = models[model].predict(test_data[x_col])

    # MSE 계산
    mse = mean_squared_error(test_data['subscribers_count'], y_pred)

    # RMSE 계산
    rmse = np.sqrt(mse)

    # R² 값 계산
    r2 = r2_score(test_data['subscribers_count'], y_pred)

    # 결과 출력
    print(model)
    print(f"R² 값: {r2:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print("----------------------------------------")
    print("")

In [92]:
# # 비선형모델 활용 변수 선정
# importances_df = pd.DataFrame({
#     'features': x_col,
#     'rf_importance': models['RandomForest'].feature_importances_,
#     'gbm_importance': models['GBM'].feature_importances_,
#     'lgbm_importance': models['LightGBM'].feature_importances_,
#     'xgb_importance': models['XGBoost'].feature_importances_    
# })
# importances_df['lgbm_importance'] = importances_df['lgbm_importance'] / importances_df['lgbm_importance'].sum() ## 다른 모델과 중요도 단위 다르기 때문에 정규화
# importances_df['mean_importance'] = importances_df[['rf_importance', 'gbm_importance', 'lgbm_importance', 'xgb_importance']].mean(axis=1) ## 각 변수별 모델 중요도 평균

In [118]:
importances_df[importances_df['mean_importance'] >= 0.003]

,features,rf_importance,gbm_importance,lgbm_importance,xgb_importance,mean_importance
1,subscribers_gained,0.000059,0.000127,0.018000,0.000332,0.004630
2,adult_viewer_rate,0.002339,0.003236,0.007667,0.000114,0.003339
3,estimatedMinutesWatched,0.000620,0.000351,0.007000,0.005589,0.003390
4,cpm,0.000424,0.030895,0.005333,0.005403,0.010514
5,subscribers_lost,0.399583,0.438425,0.152333,0.110012,0.275088
7,net_subscribers_change,0.009071,0.043670,0.031000,0.038233,0.030493
8,age13-17.female,0.003192,0.007391,0.007333,0.000139,0.004514
10,watched_time_rate,0.000514,0.000153,0.017000,0.000159,0.004456
11,age18-24.female,0.003608,0.002539,0.020333,0.007713,0.008548
12,gross_revenue_per_ad_impression,0.001233,0.016661,0.003333,0.000045,0.005318


### 모델 성능 개선

스태킹

In [27]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

# 스태킹 모델 정의
stacking_model = StackingRegressor(
    estimators=[
    ('random_forest', models['RandomForest']),
    ('gbm', models['GBM']),
    ('lightgbm', models['LightGBM']),
    ('xgboost', models['XGBoost'])
    ],
    final_estimator=Ridge()
)

# 스태킹 모델 학습
stacking_model.fit(train_data[x_col], train_data['subscribers_count'])

# 예측
y_pred = stacking_model.predict(test_data[x_col])

# 성능 평가
rmse = mean_squared_error(test_data['subscribers_count'], y_pred, squared=False)
print(f"스태킹 앙상블 모델 RMSE: {rmse:.4f}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006533 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6852
[LightGBM] [Info] Number of data points in the train set: 84473, number of used features: 27
[LightGBM] [Info] Start training from score 65433.322872
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003637 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6850
[LightGBM] [Info] Number of data points in the train set: 67578, number of used features: 27
[LightGBM] [Info] Start training from score 75989.393826
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005054 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is n

c:\Users\kwan\AppData\Local\anaconda3\envs\awake_venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### 실제데이터 결과 확인

In [66]:
# 각 모델의 예측 확률을 평균내어 결합
ensemble_probs = [model.predict_proba(merge_df_users_fin[final_selected_features]) for model in models]
ensemble_avg_probs = np.mean(np.array(ensemble_probs), axis=0)

# 최종 확률에 따른 예측값 도출
y_pred_ensemble = np.argmax(ensemble_avg_probs, axis=1)

In [86]:
# 예측 결과 확인
merge_df_users_fin['predict'] = y_pred_ensemble
pred_result_df = merge_df_users_fin.groupby(['youtube_user_id'])['predict'].value_counts().reset_index()
fraud_user_id = pred_result_df[pred_result_df['predict']==0][pred_result_df[pred_result_df['predict']==0]['count']>=40]['youtube_user_id'] ## 이상치라고 생각되는 계정 확인

In [98]:
len(set(merge_df_users_fin['youtube_user_id'].unique()) - set(merge_df_users_fin[merge_df_users_fin['youtube_user_id'].isin(fraud_user_id)]['youtube_user_id'].unique()))

207

In [90]:
set(merge_df_users_fin[merge_df_users_fin['youtube_user_id'].isin(fraud_user_id)]['channel_title'].unique())

array(['임삐나', nan, 'Mind Patting마음토닥', 'MINLEE 민리', '시골낭만아재',
       'OBL - 온라인 농부, 사자가 되다', 'abbapraise 아바프레이즈', '채림처럼firstcherry',
       '모하지연 MOHAJIYEON', 'Jeffreyxking', '콜드쉽 Coldsheep', 'kiu기우쌤',
       '수빙수tv sooBingsoo', '석시원 커플 SeokSiWon Couple', '너굴몬',
       'GMENCY 멘시의 마인크래프트', '앙찡', '코인덕 차트아지', '미니멀영어 Minimal English',
       '프롬수지 fromsuzy', 'fromsuzy 프롬수지', '日本ジヌ【니혼지누】ー韓国に関する全て', '뻘짓연구소',
       '벽돌할아버지 Brick grandpa', '북토크', '나연이즈백 LPGA Na Yeon Choi', '그롬마쉬TV',
       '뷰드름 유튜버 인씨', 'MerryMa 메리마', '쿜쿜쿜', "루다의 댄스 연구소 Ruda's Dance Lab",
       '빅민 GAME', '잼스기타', '축구 읽어주는 여자 쵱내', 'OSSC', '뛰뛰빵빵 김옥순', '돈냄새',
       '키나kkina', '이현우의 MLBTV', 'V I N 빈 ', '평범한 사업가', '하원장 강동현', '주피코',
       '목소리 연기자 유지컬'], dtype=object)

In [92]:
# 피처 중요도 추출
final_importances = xgb_model.feature_importances_
feature_importances_final = pd.DataFrame({'feature': final_selected_features, 'importance': final_importances})
feature_importances_final = feature_importances_final.sort_values(by='importance', ascending=False)

In [93]:
feature_importances_final

,feature,importance
0,estimated_revenue,0.551478
2,subscribers_gained,0.084448
6,revenue_per_minute_watched,0.073407
5,positive_engage_rate,0.070522
4,revenue_per_red_view,0.031695
1,estimatedMinutesWatched,0.014611
18,redViews,0.013696
23,subscribers_conversion_rate,0.011824
31,unsubscribed_view_time_rate,0.011104
16,monetized_playbacks,0.010411


## 콘텐츠 데이터 분석

In [5]:
# 최종 콘텐츠 분석 데이터셋
# 환율 적용 - 해당기간 평균환율 : 1322.42
exchange_rate_col = ['estimatedRevenue', 'estimatedAdRevenue', 'estimatedRedPartnerRevenue', 'grossRevenue', 'cpm', 'playbackBasedCpm']
youtube_videos[exchange_rate_col] = youtube_videos[exchange_rate_col] * 1322.42

In [6]:
# 잘못된값 처리
youtube_videos['estimatedRevenue'] = np.where(youtube_videos['estimatedRevenue'] < 0,
                                               youtube_videos['estimatedAdRevenue'] + youtube_videos['estimatedRedPartnerRevenue'],
                                               youtube_videos['estimatedRevenue'])

In [7]:
# 버그로 사용된 수치값 대체
youtube_videos['likes'] = np.where(youtube_videos['likes'] < 0, 0, youtube_videos['likes'])
youtube_videos['dislikes'] = np.where(youtube_videos['dislikes'] < 0, 0, youtube_videos['dislikes'])

### 파생변수

In [8]:
# 파생변수1 - 참여도 관련
youtube_videos['like_rate'] = youtube_videos['likes'] / youtube_videos['views'] ## 좋아요 비율 
youtube_videos['comment_rate'] = youtube_videos['comments'] / youtube_videos['views'] ## 댓글 비율
youtube_videos['share_rate'] = youtube_videos['shares'] / youtube_videos['views'] ## 공유 비율  
youtube_videos['dislike_rate'] = youtube_videos['dislikes'] / youtube_videos['views'] ## 싫어요 비율
youtube_videos['total_engage_rate'] = (youtube_videos['likes'] + youtube_videos['comments'] + youtube_videos['shares'] + youtube_videos['dislikes']) / youtube_videos['views'] ## 총 참여율
youtube_videos['positive_engage_rate'] = (youtube_videos['likes'] + youtube_videos['shares']) / youtube_videos['views'] ## 긍정적 참여율
youtube_videos['comment_to_like_rate'] = youtube_videos['comments'] / youtube_videos['likes'] ## 댓글/좋아요 비율
youtube_videos['like_to_dislike_ratio'] = youtube_videos['likes'] / (youtube_videos['dislikes']) ## 좋아요/싫어요 비율

In [9]:
# 파생변수2 - 구독자 관련
youtube_videos['net_subscribers_change'] = youtube_videos['subscribersGained'] - youtube_videos['subscribersLost'] ## 구독자 순증가
youtube_videos['subscribers_conversion_rate'] = youtube_videos['subscribersGained'] / youtube_videos['views'] ## 구독자 전환율
youtube_videos['subscribers_gained_per_card_click'] = youtube_videos['subscribersGained'] / youtube_videos['cardClicks'] ## 카드 클릭당 구독자 증가
youtube_videos['subscribers_gained_per_playlist_add'] = youtube_videos['subscribersGained'] / youtube_videos['videosAddedToPlaylists'] ## 플레이리스트 추가당 구독자 증가
youtube_videos['card_click_to_subscriber_conversion'] = youtube_videos['cardClickRate'] / youtube_videos['subscribersGained'] ## 카드 클릭률 대비 구독자 전환율
youtube_videos['subscribers_lost_per_playlist_remove'] = youtube_videos['subscribersLost'] / youtube_videos['videosRemovedFromPlaylists'] ## 플레이리스트 제거당 구독자 손실

In [10]:
# 파생변수3 - 수익 관련
youtube_videos['revenue_per_view'] = youtube_videos['estimatedRevenue'] / youtube_videos['views'] ## 조회수당 수익
youtube_videos['revenue_per_red_view'] = youtube_videos['estimatedRevenue'] / youtube_videos['redViews'] ## 프리미엄당 수익
youtube_videos['ad_revenue_rate'] = youtube_videos['estimatedAdRevenue'] / youtube_videos['estimatedRevenue'] ## 광고수익비율
youtube_videos['red_revenue_rate'] = youtube_videos['estimatedRedPartnerRevenue'] / youtube_videos['estimatedRevenue'] ## 프리미엄수익비율
youtube_videos['cpm_to_revenue_ratio'] = youtube_videos['cpm'] / youtube_videos['estimatedRevenue'] ## cpm 대비 수익
youtube_videos['revenue_per_ad_impression'] = youtube_videos['estimatedRevenue'] / youtube_videos['adImpressions'] ## 광고노출당 수익
youtube_videos['playback_based_cpm_rate'] = youtube_videos['playbackBasedCpm'] / youtube_videos['cpm'] ## 재생 기반 수익
youtube_videos['revenue_per_card_click'] = youtube_videos['estimatedRevenue'] / youtube_videos['cardClicks'] ## 카드 클릭당 수익
youtube_videos['revenue_per_playlist_add'] = youtube_videos['estimatedRevenue'] / youtube_videos['videosAddedToPlaylists'] ## 플레이리스트 추가당 수익
youtube_videos['card_click_to_revenue_ratio'] = youtube_videos['cardClickRate'] / youtube_videos['estimatedRevenue'] ## 카드 클릭률 대비 수익 비율
youtube_videos['net_revenue_per_playlist_add'] = (youtube_videos['videosAddedToPlaylists'] - youtube_videos['videosRemovedFromPlaylists']) / youtube_videos['estimatedRevenue'] ## 플레이리스트 순추가당 수익
youtube_videos['ad_revenue_per_card_click'] = youtube_videos['estimatedAdRevenue'] / youtube_videos['cardClicks'] ## 카드 클릭당 광고 수익

In [11]:
# 파생변수4 - 시청 시간 관련
youtube_videos['revenue_per_minute_watched'] = youtube_videos['estimatedRevenue'] / youtube_videos['estimatedMinutesWatched'] ## 시청 시간 당 수익
youtube_videos['revenue_per_red_minute_watched'] = youtube_videos['estimatedRevenue'] / youtube_videos['estimatedRedMinutesWatched'] ## 프리미엄 이용자 시청 시간 당 수익
youtube_videos['avg_view_duration_rate'] = youtube_videos['averageViewDuration'] / youtube_videos['averageViewPercentage'] ## 평균 시청 시간 비율
youtube_videos['watched_time_rate'] = youtube_videos['averageViewPercentage'] * youtube_videos['estimatedMinutesWatched'] ## 재생 비율 대비 시청 시간
youtube_videos['watched_red_time_rate'] = youtube_videos['averageViewPercentage'] * youtube_videos['estimatedRedMinutesWatched'] ## 재생 비율 대비 프리미엄 이용자 시청 시간
youtube_videos['watched_view_rate'] = youtube_videos['estimatedMinutesWatched'] / youtube_videos['views'] ## 조회수당 시청시간
youtube_videos['watched_red_view_rate'] = youtube_videos['estimatedRedMinutesWatched'] / youtube_videos['views'] ## 조회수당 프리미엄 이용자 시청시간
youtube_videos['watch_time_per_card_click'] = youtube_videos['estimatedMinutesWatched'] / youtube_videos['cardClicks'] ## 카드 클릭당 시청 시간
youtube_videos['watch_time_per_playlist_add'] = youtube_videos['estimatedMinutesWatched'] / youtube_videos['videosAddedToPlaylists'] ## 플레이리스트 추가당 시청 시간
youtube_videos['avg_view_duration_per_card_click'] = youtube_videos['averageViewDuration'] / youtube_videos['cardClicks'] ## 카드 클릭 대비 평균 재생 시간 비율
youtube_videos['watch_time_loss_per_playlist_remove'] = youtube_videos['estimatedMinutesWatched'] / youtube_videos['videosRemovedFromPlaylists'] ## 플레이리스트 제거당 시청 시간 손실


In [12]:
# 파생변수5 - 광고 관련
youtube_videos['revenue_per_playback'] = youtube_videos['grossRevenue'] / youtube_videos['monetizedPlaybacks'] ## 1회 광고재생당 수익
youtube_videos['grossRevenue_per_ad_impression'] = youtube_videos['grossRevenue'] / youtube_videos['adImpressions'] ## 1회 광고노출당 총수익
youtube_videos['playback_rate'] = youtube_videos['monetizedPlaybacks'] / youtube_videos['adImpressions'] ## 광고노출 대비 재생율
youtube_videos['unplayback_rate'] = (youtube_videos['adImpressions'] - youtube_videos['monetizedPlaybacks']) / youtube_videos['adImpressions'] ## 광고노출 대비 비재생율
youtube_videos['grossrevenue_per_card_click'] = youtube_videos['grossRevenue'] / youtube_videos['cardClicks'] ## 카드 클릭당 수익
youtube_videos['grossrevenue_per_playlist_add'] = youtube_videos['grossRevenue'] / youtube_videos['videosAddedToPlaylists'] ## 플레이리스트 추가당 수익
youtube_videos['card_click_to_grossrevenue_ratio'] = youtube_videos['cardClickRate'] / youtube_videos['grossRevenue'] ## 카드 클릭률 대비 수익 비율
youtube_videos['net_grossrevenue_per_playlist_add'] = (youtube_videos['videosAddedToPlaylists'] - youtube_videos['videosRemovedFromPlaylists']) / youtube_videos['grossRevenue'] ## 플레이리스트 순추가당 수익
youtube_videos['ad_impressions_per_card_click'] = youtube_videos['adImpressions'] / youtube_videos['cardClicks'] ## 카드 클릭당 광고 노출
youtube_videos['ad_impressions_per_playlist_add'] = youtube_videos['adImpressions'] / youtube_videos['videosAddedToPlaylists'] ## 플레이리스트 추가당 광고 노출
youtube_videos['ad_playbacks_per_card_click'] = youtube_videos['monetizedPlaybacks'] / youtube_videos['cardClicks'] ## 카드 클릭 대비 광고 재생 비율
youtube_videos['ad_playbacks_per_playlist_add'] = youtube_videos['monetizedPlaybacks'] / youtube_videos['videosAddedToPlaylists'] ## 플레이리스트 추가당 광고 재생 비율

In [13]:
# 파생변수6 - 카드 관련
youtube_videos['card_to_teaser_click_rate'] = youtube_videos['cardClickRate'] / youtube_videos['cardTeaserClickRate'] ## 카드 티저 클릭률 대비 카드 클릭률
youtube_videos['card_click_per_impression_rate'] = youtube_videos['cardClicks'] / youtube_videos['cardImpressions'] ## 카드 노출당 클릭 비율
youtube_videos['card_teaser_click_per_impression_rate'] = youtube_videos['cardTeaserClicks'] / youtube_videos['cardTeaserImpressions'] ## 카드 티저 노출당 카드 티저 클릭 비율
youtube_videos['total_card_teaser_click_rate'] = (youtube_videos['cardClicks'] + youtube_videos['cardTeaserClicks']) / (youtube_videos['cardImpressions'] + youtube_videos['cardTeaserImpressions']) ## 카드와 카드 티저의 총 클릭률
youtube_videos['card_conversion_rate'] = youtube_videos['cardClicks'] / youtube_videos['cardTeaserClicks'] ## 카드 클릭 전환율

In [14]:
# 파생변수7 - 비디오 관련
youtube_videos['playlist_addition_rate'] = youtube_videos['videosAddedToPlaylists'] / youtube_videos['views'] ## 플레이리스트 추가 비율
youtube_videos['playlist_removal_rate'] = youtube_videos['videosRemovedFromPlaylists'] / youtube_videos['views'] ## 플레이리스트 제거 비율
youtube_videos['net_playlist_addition_rate'] = (youtube_videos['videosAddedToPlaylists'] - youtube_videos['videosRemovedFromPlaylists']) / youtube_videos['views'] ## 플레이리스트 순추가 비율
youtube_videos['playlist_engagement_rate'] = (youtube_videos['videosAddedToPlaylists'] + youtube_videos['videosRemovedFromPlaylists']) / youtube_videos['views'] ## 플레이리스트 참여도
youtube_videos['playlist_related_revenue_rate'] = youtube_videos['estimatedRevenue'] / youtube_videos['videosAddedToPlaylists'] ## 플레이리스트 추가 당 순수익

In [16]:
# null값 대체
youtube_videos[youtube_videos.columns[3:]] = youtube_videos[youtube_videos.columns[3:]].fillna(0) ## NaN
youtube_videos[youtube_videos.columns[3:]] = np.where(youtube_videos[youtube_videos.columns[3:]]==np.inf,0,youtube_videos[youtube_videos.columns[3:]]) ## inf

### y값 설정

#### 중요 지표 표준화

In [22]:
# y값 활용 지표
y_col = ['total_engage_rate', 'net_subscribers_change', 'averageViewPercentage', 'revenue_per_view', 'grossRevenue_per_ad_impression','total_card_teaser_click_rate','playlist_engagement_rate'] ## null값은 views가 0인 데이터

In [23]:
# null값 대체
youtube_videos[y_col] = youtube_videos[y_col].fillna(0) ## NaN
youtube_videos[y_col] = np.where(youtube_videos[y_col]==np.inf,0,youtube_videos[y_col]) ## inf

In [26]:
# 표준화 (Standard Scaling)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(youtube_videos[y_col])

#### 다중 지표 결합

In [27]:
# Isolation Forest 모델 학습
iso_forest = IsolationForest(contamination=0.05, random_state=42)  # 이상치 비율 설정
iso_forest.fit(scaled_features)

# 이상치 점수 계산
anomaly_scores = iso_forest.decision_function(scaled_features)

In [28]:
# 임계값 설정
threshold = np.percentile(anomaly_scores, 5) ## 이상치 점수가 작을수록 이상치에 가깝다고 판단하는데, 5번째 백분위수가 (-)로 바뀌는 포인트

# y값 라벨링 (이상치이면 0, 그렇지 않으면 1)
youtube_videos['y_label'] = np.where(anomaly_scores < threshold, 0, 1)

In [29]:
# 전체 데이터 y값 빈도 확인
youtube_videos['y_label'].value_counts()

y_label
1    8141038
0     428473
Name: count, dtype: int64

In [30]:
del scaled_features, anomaly_scores

### 데이터 분할

In [31]:
# 컬럼 정리
unique_col = youtube_videos.columns[:3]
x_col = youtube_videos.columns[3:-1]

In [35]:
# 데이터 분할
X = youtube_videos[x_col].drop(columns=y_col) ## y값 라벨링에 쓰인 지표 제거
y = youtube_videos['y_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
del X, y

In [37]:
# 불균형 확인
print(y_train.value_counts())
print(y_test.value_counts())

y_label
1    6513105
0     342503
Name: count, dtype: int64
y_label
1    1627933
0      85970
Name: count, dtype: int64


### 언더샘플링

In [38]:
# 언더샘플링
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

### 변수선택

t-test

In [39]:
from scipy.stats import ttest_ind

# y=0과 y=1 그룹으로 데이터 나누기
group_0 = X_train_resampled[y_train_resampled == 0]
group_1 = X_train_resampled[y_train_resampled == 1]

# t-test 수행
p_values = []
for col in X_train_resampled.columns:
    t_stat, p_val = ttest_ind(group_0[col], group_1[col], equal_var=False)  # Welch's t-test
    p_values.append((col, p_val))

# p-value < 0.05인 변수 선택
selected_features_by_ttest = [col for col, p_val in p_values if p_val < 0.05]
print("Selected Features by t-test:", selected_features_by_ttest)

Selected Features by t-test: ['views', 'redViews', 'comments', 'likes', 'dislikes', 'shares', 'estimatedMinutesWatched', 'estimatedRedMinutesWatched', 'averageViewDuration', 'videosAddedToPlaylists', 'videosRemovedFromPlaylists', 'estimatedRevenue', 'estimatedAdRevenue', 'grossRevenue', 'estimatedRedPartnerRevenue', 'playbackBasedCpm', 'cpm', 'subscribersGained', 'subscribersLost', 'monetizedPlaybacks', 'adImpressions', 'cardClickRate', 'cardTeaserClickRate', 'cardImpressions', 'cardTeaserImpressions', 'cardClicks', 'cardTeaserClicks', 'like_rate', 'comment_rate', 'share_rate', 'dislike_rate', 'positive_engage_rate', 'comment_to_like_rate', 'like_to_dislike_ratio', 'subscribers_conversion_rate', 'subscribers_gained_per_card_click', 'subscribers_gained_per_playlist_add', 'card_click_to_subscriber_conversion', 'subscribers_lost_per_playlist_remove', 'revenue_per_red_view', 'ad_revenue_rate', 'red_revenue_rate', 'cpm_to_revenue_ratio', 'revenue_per_ad_impression', 'playback_based_cpm_rate

c:\Users\kkims\anaconda3\envs\awake\lib\site-packages\scipy\stats\_stats_py.py:1079: RuntimeWarning: invalid value encountered in subtract
  a_zero_mean = a - mean
c:\Users\kkims\anaconda3\envs\awake\lib\site-packages\scipy\stats\_stats_py.py:6527: RuntimeWarning: invalid value encountered in scalar subtract
  d = mean1 - mean2
c:\Users\kkims\anaconda3\envs\awake\lib\site-packages\scipy\stats\_stats_py.py:7025: RuntimeWarning: invalid value encountered in scalar subtract
  estimate = m1-m2


In [47]:
len(selected_features_by_ttest)

74

Lasso

In [48]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

# t-test로 선택된 변수로 데이터프레임 구성
X_train_ttest_selected = X_train_resampled[selected_features_by_ttest]

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_ttest_selected)

# LassoCV 모델 설정: 반복 횟수 증가, alpha 범위 조정
lasso = LassoCV(cv=5, random_state=42, max_iter=10000, alphas=[0.1, 0.05, 0.01, 0.005, 0.001]).fit(X_train_scaled, y_train_resampled)

# 선택된 변수 확인 (회귀 계수가 0이 아닌 변수들)
selected_features_by_lasso = X_train_ttest_selected.columns[(lasso.coef_ != 0)]
print("Selected Features by Lasso:", selected_features_by_lasso)

Selected Features by Lasso: Index(['views', 'redViews', 'comments', 'likes', 'dislikes', 'shares',
       'estimatedRedMinutesWatched', 'averageViewDuration',
       'videosAddedToPlaylists', 'videosRemovedFromPlaylists',
       'estimatedRevenue', 'estimatedAdRevenue', 'grossRevenue',
       'playbackBasedCpm', 'cpm', 'subscribersGained', 'subscribersLost',
       'monetizedPlaybacks', 'adImpressions', 'cardClickRate',
       'cardTeaserClickRate', 'cardImpressions', 'cardTeaserImpressions',
       'like_rate', 'comment_rate', 'dislike_rate', 'positive_engage_rate',
       'comment_to_like_rate', 'like_to_dislike_ratio',
       'subscribers_conversion_rate', 'subscribers_gained_per_playlist_add',
       'revenue_per_red_view', 'ad_revenue_rate', 'red_revenue_rate',
       'cpm_to_revenue_ratio', 'revenue_per_ad_impression',
       'playback_based_cpm_rate', 'revenue_per_playlist_add',
       'revenue_per_minute_watched', 'avg_view_duration_rate',
       'watched_time_rate', 'watched_r

In [50]:
len(selected_features_by_lasso)

55

In [53]:
## lasso 선택으로 제거된 변수
set(selected_features_by_ttest) - set(selected_features_by_lasso)

{'ad_impressions_per_card_click',
 'ad_impressions_per_playlist_add',
 'ad_playbacks_per_card_click',
 'ad_revenue_per_card_click',
 'cardClicks',
 'cardTeaserClicks',
 'card_click_to_subscriber_conversion',
 'card_to_teaser_click_rate',
 'estimatedMinutesWatched',
 'estimatedRedPartnerRevenue',
 'grossrevenue_per_card_click',
 'revenue_per_card_click',
 'revenue_per_red_minute_watched',
 'share_rate',
 'subscribers_gained_per_card_click',
 'subscribers_lost_per_playlist_remove',
 'watch_time_per_card_click',
 'watched_red_time_rate',
 'watched_view_rate'}

RandomForest

In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# RandomForest 모델 학습 및 교차 검증
X_train_lasso_selected = X_train_resampled[selected_features_by_lasso]

model = RandomForestClassifier(random_state=42)
cross_val_scores = cross_val_score(model, X_train_lasso_selected, y_train_resampled, cv=5, scoring='accuracy')

# 모델 학습
model.fit(X_train_lasso_selected, y_train_resampled)

# 피처 중요도 추출
importances = model.feature_importances_
feature_importances = pd.DataFrame({'feature': selected_features_by_lasso, 'importance': importances})
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# 최종적으로 중요한 변수 선택 (예: 상위 10개 변수)
final_selected_features = list(feature_importances[feature_importances['importance'] > 0.005]['feature'])
print("Final Selected Features by RandomForest:", final_selected_features)

# 교차 검증 결과 출력
print(f"Cross-Validation Accuracy: {cross_val_scores.mean():.2f}")

Final Selected Features by RandomForest: ['positive_engage_rate', 'playlist_addition_rate', 'cpm', 'revenue_per_ad_impression', 'revenue_per_playback', 'playlist_removal_rate', 'like_rate', 'playbackBasedCpm', 'videosAddedToPlaylists', 'views', 'likes', 'videosRemovedFromPlaylists', 'subscribersGained', 'watch_time_per_playlist_add', 'shares', 'watched_time_rate', 'revenue_per_red_view', 'estimatedRevenue', 'redViews', 'averageViewDuration', 'revenue_per_minute_watched', 'watch_time_loss_per_playlist_remove', 'avg_view_duration_rate', 'revenue_per_playlist_add', 'estimatedAdRevenue', 'watched_red_view_rate', 'estimatedRedMinutesWatched', 'grossRevenue', 'cpm_to_revenue_ratio', 'comment_rate', 'playlist_related_revenue_rate', 'ad_revenue_rate', 'comments', 'subscribers_conversion_rate', 'red_revenue_rate', 'ad_playbacks_per_playlist_add', 'monetizedPlaybacks', 'adImpressions']
Cross-Validation Accuracy: 0.99


In [58]:
final_selected_features

['positive_engage_rate',
 'playlist_addition_rate',
 'cpm',
 'revenue_per_ad_impression',
 'revenue_per_playback',
 'playlist_removal_rate',
 'like_rate',
 'playbackBasedCpm',
 'videosAddedToPlaylists',
 'views',
 'likes',
 'videosRemovedFromPlaylists',
 'subscribersGained',
 'watch_time_per_playlist_add',
 'shares',
 'watched_time_rate',
 'revenue_per_red_view',
 'estimatedRevenue',
 'redViews',
 'averageViewDuration',
 'revenue_per_minute_watched',
 'watch_time_loss_per_playlist_remove',
 'avg_view_duration_rate',
 'revenue_per_playlist_add',
 'estimatedAdRevenue',
 'watched_red_view_rate',
 'estimatedRedMinutesWatched',
 'grossRevenue',
 'cpm_to_revenue_ratio',
 'comment_rate',
 'playlist_related_revenue_rate',
 'ad_revenue_rate',
 'comments',
 'subscribers_conversion_rate',
 'red_revenue_rate',
 'ad_playbacks_per_playlist_add',
 'monetizedPlaybacks',
 'adImpressions']

In [56]:
feature_importances ## 0.005 기준으로 변수 선택

,feature,importance
26,positive_engage_rate,0.115805
52,playlist_addition_rate,0.072534
14,cpm,0.066794
35,revenue_per_ad_impression,0.056216
45,revenue_per_playback,0.054511
53,playlist_removal_rate,0.049415
23,like_rate,0.049097
13,playbackBasedCpm,0.048472
8,videosAddedToPlaylists,0.045500
0,views,0.033676


### 모델 기법 적용

RandomForest

In [59]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤 포레스트 모델 학습
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_resampled[final_selected_features], y_train_resampled)

# # 랜덤 포레스트 모델 교차 검증
# cv_scores_rf = cross_val_score(rf_model, X_train_resampled[final_selected_features], y_train_resampled, cv=5, scoring='accuracy')
# print(f"RandomForest Cross-Validation Accuracy: {cv_scores_rf.mean():.2f}")

RandomForestClassifier(random_state=42)

GradientBoosting

In [60]:
from sklearn.ensemble import GradientBoostingClassifier

# 그라디언트 부스팅 모델 학습
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train_resampled[final_selected_features], y_train_resampled)

# # 그라디언트 부스팅 모델 교차 검증
# cv_scores_gb = cross_val_score(gb_model, X_train_resampled[final_selected_features], y_train_resampled, cv=5, scoring='accuracy')
# print(f"GradientBoosting Cross-Validation Accuracy: {cv_scores_gb.mean():.2f}")

GradientBoostingClassifier(random_state=42)

XGBoost

In [61]:
from xgboost import XGBClassifier

# XGBoost를 사용한 모델 학습
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train_resampled[final_selected_features], y_train_resampled)

# # 그라디언트 부스팅 모델 교차 검증
# cv_scores_xgb = cross_val_score(xgb_model, X_train_resampled[final_selected_features], y_train_resampled, cv=5, scoring='accuracy')
# print(f"GradientBoosting Cross-Validation Accuracy: {cv_scores_xgb.mean():.2f}")

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

### 모델 성능 평가

In [62]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

# 예측 결과 생성
y_pred_rf = rf_model.predict(X_test[final_selected_features])
y_pred_gb = gb_model.predict(X_test[final_selected_features])
y_pred_xgb = xgb_model.predict(X_test[final_selected_features])

# 모델 정확도 확인
accuracy_rf = accuracy_score(y_test, y_pred_rf)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)

print(f"RandomForest Test Accuracy: {accuracy_rf:.2f}")
print(f"GradientBoosting Test Accuracy: {accuracy_gb:.2f}")
print(f"XGBoost Test Accuracy: {accuracy_xgb:.2f}")

# 정밀도, 재현율, F1-score 출력
print("RandomForest Classification Report:")
print(classification_report(y_test, y_pred_rf))

print("GradientBoosting Classification Report:")
print(classification_report(y_test, y_pred_gb))

print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))

# ROC-AUC 확인
roc_auc_rf = roc_auc_score(y_test, rf_model.predict_proba(X_test[final_selected_features])[:, 1])
roc_auc_gb = roc_auc_score(y_test, gb_model.predict_proba(X_test[final_selected_features])[:, 1])
roc_auc_xgb = roc_auc_score(y_test, xgb_model.predict_proba(X_test[final_selected_features])[:, 1])

print(f"RandomForest ROC-AUC: {roc_auc_rf:.2f}")
print(f"GradientBoosting ROC-AUC: {roc_auc_gb:.2f}")
print(f"XGBoost ROC-AUC: {roc_auc_xgb:.2f}")

RandomForest Test Accuracy: 0.98
GradientBoosting Test Accuracy: 0.96
XGBoost Test Accuracy: 0.99
RandomForest Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.99      0.84     85970
           1       1.00      0.98      0.99   1627933

    accuracy                           0.98   1713903
   macro avg       0.87      0.99      0.92   1713903
weighted avg       0.99      0.98      0.98   1713903

GradientBoosting Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.98      0.74     85970
           1       1.00      0.96      0.98   1627933

    accuracy                           0.96   1713903
   macro avg       0.79      0.97      0.86   1713903
weighted avg       0.98      0.96      0.97   1713903

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.99      0.90     85970
           1       1.00      0.9

### 모델 성능 개선

언더샘플링 데이터셋별 각 모델 학습

In [40]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
import numpy as np

# 언더샘플링 횟수 설정
n_iterations = 5

# 각 모델의 예측 확률 저장 리스트
rf_probs = []
gb_probs = []
xgb_probs = []
models = []

for i in range(n_iterations):
    # 언더샘플링 적용
    rus = RandomUnderSampler(random_state=i)  # 각기 다른 랜덤 시드 사용
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

    # 랜덤포레스트 모델 학습
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train_resampled[final_selected_features], y_train_resampled)
    rf_probs.append(rf_model.predict_proba(X_test[final_selected_features]))
    models.append(rf_model)

    # 그라디언트 부스팅 모델 학습
    gb_model = GradientBoostingClassifier(random_state=42)
    gb_model.fit(X_train_resampled[final_selected_features], y_train_resampled)
    gb_probs.append(gb_model.predict_proba(X_test[final_selected_features]))
    models.append(gb_model)

    # XGBoost 모델 학습
    xgb_model = XGBClassifier(random_state=42)
    xgb_model.fit(X_train_resampled[final_selected_features], y_train_resampled)
    xgb_probs.append(xgb_model.predict_proba(X_test[final_selected_features]))
    models.append(xgb_model)

In [ ]:
models

In [41]:
# 확률 평균을 통한 앙상블
rf_probs_avg = np.mean(np.array(rf_probs), axis=0)
gb_probs_avg = np.mean(np.array(gb_probs), axis=0)
xgb_probs_avg = np.mean(np.array(xgb_probs), axis=0)

# 최종 평균 확률 계산
ensemble_probs_avg = (rf_probs_avg + gb_probs_avg + xgb_probs_avg) / 3

# 평균 확률에 따라 최종 예측
y_pred_ensemble_avg = np.argmax(ensemble_probs_avg, axis=1)

# 성능 평가
from sklearn.metrics import accuracy_score
accuracy_ensemble_avg = accuracy_score(y_test, y_pred_ensemble_avg)
print(f"Ensemble Averaging Accuracy: {accuracy_ensemble_avg:.2f}")


Ensemble Averaging Accuracy: 0.98


In [45]:
from sklearn.metrics import classification_report

# 각 모델의 클래스별 성능 평가
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf, target_names=['Class 0', 'Class 1']))

print("\nGradient Boosting Classification Report:")
print(classification_report(y_test, y_pred_gb, target_names=['Class 0', 'Class 1']))

print("\nXGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb, target_names=['Class 0', 'Class 1']))

print("\nEnsemble Model Classification Report:")
print(classification_report(y_test, y_pred_ensemble_avg, target_names=['Class 0', 'Class 1']))


Random Forest Classification Report:
              precision    recall  f1-score   support

     Class 0       0.70      0.99      0.82     85970
     Class 1       1.00      0.98      0.99   1627933

    accuracy                           0.98   1713903
   macro avg       0.85      0.99      0.90   1713903
weighted avg       0.98      0.98      0.98   1713903


Gradient Boosting Classification Report:
              precision    recall  f1-score   support

     Class 0       0.52      0.96      0.68     85970
     Class 1       1.00      0.95      0.98   1627933

    accuracy                           0.95   1713903
   macro avg       0.76      0.96      0.83   1713903
weighted avg       0.97      0.95      0.96   1713903


XGBoost Classification Report:
              precision    recall  f1-score   support

     Class 0       0.79      1.00      0.88     85970
     Class 1       1.00      0.99      0.99   1627933

    accuracy                           0.99   1713903
   macro avg     

### 실제데이터 결과 확인

In [49]:
# 각 모델의 예측 확률을 평균내어 결합
ensemble_probs = [model.predict_proba(youtube_videos[final_selected_features]) for model in models]
ensemble_avg_probs = np.mean(np.array(ensemble_probs), axis=0)

# 최종 확률에 따른 예측값 도출
y_pred_ensemble = np.argmax(ensemble_avg_probs, axis=1)

In [50]:
# 예측 결과 확인
youtube_videos['predict'] = y_pred_ensemble

In [54]:
youtube_videos[youtube_videos['predict']==0].groupby('video').aggregate()

,youtube_user_id,video,end_date,views,redViews,comments,likes,dislikes,shares,estimatedMinutesWatched,...,card_teaser_click_per_impression_rate,total_card_teaser_click_rate,card_conversion_rate,playlist_addition_rate,playlist_removal_rate,net_playlist_addition_rate,playlist_engagement_rate,playlist_related_revenue_rate,y_label,predict
70,6401da7ad746c60e1271fdd6,--0XOlJ3Lw4,2023-05-26,39,7,1,0,0,0,202,...,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.000000,inf,1,0
72,6401da7ad746c60e1271fdd6,--0XOlJ3Lw4,2023-05-28,44,15,0,0,0,0,220,...,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.000000,inf,1,0
100,6401da7ad746c60e1271fdd6,--0XOlJ3Lw4,2023-06-25,21,6,0,1,0,0,148,...,NaN,0.0,NaN,0.095238,0.095238,0.000000,0.190476,28.43203,1,0
101,6401da7ad746c60e1271fdd6,--0XOlJ3Lw4,2023-06-26,28,6,0,2,0,0,139,...,NaN,0.0,NaN,0.071429,0.107143,-0.035714,0.178571,51.57438,1,0
144,6401da7ad746c60e1271fdd6,--0XOlJ3Lw4,2023-08-08,64,20,3,0,0,0,372,...,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.000000,inf,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8569349,63dbc0acee122e631991ca5b,zzza6bbJnMI,2023-07-14,7,1,0,0,0,0,74,...,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.000000,inf,0,0
8569350,63dbc0acee122e631991ca5b,zzza6bbJnMI,2023-07-15,7,2,0,0,0,0,49,...,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.000000,inf,1,0
8569410,63dbc0acee122e631991ca5b,zzza6bbJnMI,2023-10-21,1,0,0,0,0,0,5,...,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.000000,NaN,1,0
8569432,63dbc0acee122e631991ca5b,zzza6bbJnMI,2023-12-10,2,0,0,0,0,1,0,...,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.000000,NaN,1,0


In [ ]:
youtube_videos

In [ ]:
set(merge_df_users_fin[merge_df_users_fin['youtube_user_id'].isin(fraud_user_id)]['channel_title'].unique())

array(['임삐나', nan, 'Mind Patting마음토닥', 'MINLEE 민리', '시골낭만아재',
       'OBL - 온라인 농부, 사자가 되다', 'abbapraise 아바프레이즈', '채림처럼firstcherry',
       '모하지연 MOHAJIYEON', 'Jeffreyxking', '콜드쉽 Coldsheep', 'kiu기우쌤',
       '수빙수tv sooBingsoo', '석시원 커플 SeokSiWon Couple', '너굴몬',
       'GMENCY 멘시의 마인크래프트', '앙찡', '코인덕 차트아지', '미니멀영어 Minimal English',
       '프롬수지 fromsuzy', 'fromsuzy 프롬수지', '日本ジヌ【니혼지누】ー韓国に関する全て', '뻘짓연구소',
       '벽돌할아버지 Brick grandpa', '북토크', '나연이즈백 LPGA Na Yeon Choi', '그롬마쉬TV',
       '뷰드름 유튜버 인씨', 'MerryMa 메리마', '쿜쿜쿜', "루다의 댄스 연구소 Ruda's Dance Lab",
       '빅민 GAME', '잼스기타', '축구 읽어주는 여자 쵱내', 'OSSC', '뛰뛰빵빵 김옥순', '돈냄새',
       '키나kkina', '이현우의 MLBTV', 'V I N 빈 ', '평범한 사업가', '하원장 강동현', '주피코',
       '목소리 연기자 유지컬'], dtype=object)

In [55]:
# 피처 중요도 추출
final_importances = xgb_model.feature_importances_
feature_importances_final = pd.DataFrame({'feature': final_selected_features, 'importance': final_importances})
feature_importances_final = feature_importances_final.sort_values(by='importance', ascending=False)

In [58]:
feature_importances_final

,feature,importance
1,videosAddedToPlaylists,0.173585
0,cpm,0.168345
4,videosRemovedFromPlaylists,0.108364
2,likes,0.099482
5,shares,0.092513
7,subscribersGained,0.064080
18,dislikes,0.057080
3,views,0.054972
17,comments,0.036138
9,redViews,0.034451
